# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Make a reference to the city data.csv file path
csv_path = "../weatherpy/city_df.csv"

# Import the city data.csv file as a DataFrame
city_df = pd.read_csv(csv_path, encoding="utf-8")
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,conceicao do araguaia,-8.2578,-49.2647,73.49,96,100,3.94,BR,1612663380
1,luderitz,-26.6481,15.1594,67.42,70,0,22.70,NaN,1612663104
2,kapaa,22.0752,-159.3190,73.40,53,75,6.91,US,1612663389
3,bethel,41.3712,-73.4140,35.01,59,1,4.74,US,1612662990
4,ambon,-3.6954,128.1814,89.60,62,40,5.75,ID,1612663039


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
#Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]].astype(float)
weights = city_df["Humidity"].astype(float)

# Create a humidity heat layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=3)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,conceicao do araguaia,-8.2578,-49.2647,73.49,96,100,3.94,BR,1612663380
1,luderitz,-26.6481,15.1594,67.42,70,0,22.70,NaN,1612663104
2,kapaa,22.0752,-159.3190,73.40,53,75,6.91,US,1612663389
3,bethel,41.3712,-73.4140,35.01,59,1,4.74,US,1612662990
4,ambon,-3.6954,128.1814,89.60,62,40,5.75,ID,1612663039


In [6]:
#Ideal weather conditions - max_temp less than 80 but higher than 70, Wind speed less 10 mph, and Zero cloudiness.  
new_df = city_df.loc[(city_df["Max Temp"] < 80) &\
                     (city_df["Max Temp"] > 70) &\
                     (city_df["Wind Speed"] < 10) &\
                     (city_df["Cloudiness"] == 0)]

new_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
154,te anau,-45.4167,167.7167,75.90,38,0,5.68,NZ,1612663298
181,yaan,7.3833,8.5667,73.72,35,0,5.48,NG,1612663985
183,presidencia roque saenz pena,-26.7852,-60.4388,73.99,66,0,1.99,AR,1612663991
257,puerto baquerizo moreno,-0.9000,-89.6000,76.73,85,0,7.78,EC,1612663941
260,general roca,-39.0333,-67.5833,75.20,69,0,2.30,AR,1612664005


In [7]:
new_df["Hotel Name"]=""  
hotelone_df = new_df[["City", "Lat", "Lng", "Country", "Hotel Name"]]
hotelone_df.head()

<ipython-input-7-49afe221142b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Hotel Name"]=""


,City,Lat,Lng,Country,Hotel Name
154,te anau,-45.4167,167.7167,NZ,
181,yaan,7.3833,8.5667,NG,
183,presidencia roque saenz pena,-26.7852,-60.4388,AR,
257,puerto baquerizo moreno,-0.9000,-89.6000,EC,
260,general roca,-39.0333,-67.5833,AR,


In [8]:
#Drop any rows with null values from hotel_df
hotel_df = hotelone_df.dropna(axis=0, how='all', inplace = False) 


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Create parameters to pull back hotels meeting criteria
params = {
    "radius": "5000",
    "type": "lodging",
    "keyword" : "hotels",
    "key": g_key}

# use iterrows to iterate through dataframe      
    
for index, row in hotel_df.iterrows():
    #find the closet hotels within a 5000 meter radious of the location
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #get latitude and longitude from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change the location to itterate through the cities
    params["locations"] = f"{lat},{lng}" 
    
    # assemble url and make API request
    hotel_response = requests.get(base_url, params=params)
    
    # convert to json
    hotel_response = hotel_response.json()
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = hotel_response["results"][0]['name']      
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")    

hotel_df


Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------


,City,Lat,Lng,Country,Hotel Name
154,te anau,-45.4167,167.7167,NZ,
181,yaan,7.3833,8.5667,NG,
183,presidencia roque saenz pena,-26.7852,-60.4388,AR,
257,puerto baquerizo moreno,-0.9000,-89.6000,EC,
260,general roca,-39.0333,-67.5833,AR,
294,longyan,25.1072,117.0225,CN,
305,mandera,3.9366,41.8670,KE,
323,puntarenas,9.9763,-84.8384,CR,
334,nong khai,17.9167,103.0833,TH,
363,phonhong,18.5007,102.4166,LA,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, infoboxcontent = hotel_info )
fig.add_layer(markers)

# Display figure
fig


TypeError: marker_layer() got an unexpected keyword argument 'infoboxcontent'